In [1]:
# numpy 2. jotain tulee condan asennuksessa, vaikka ei näy listauksessa, tämä koodi downgradeaa sen

# import pip
# pip.main(['install', 'numpy<2'])
# import numpy as np
# print(np.__version__)

### Surprise sopii käyttäjien arvioiden perusteella suosittelun ennustamiseen

https://surprise.readthedocs.io/en/stable/

In [2]:
# # (conda_env) U:\Users\ok025980>conda install -n conda_env -c conda-forge scikit-surprise
from surprise import Reader
from surprise import Dataset

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import KNNBasic
from surprise.model_selection import cross_validate
import pprint

In [4]:
# Luodaan dataframe dictionarysta pandas

# dictionaryn luominen
ratings = {'itemid': ['Tuote1', 'Tuote2', 'Tuote3', 'Tuote4', 'Tuote1', 'Tuote2', 'Tuote3', 'Tuote4', 'Tuote1', 'Tuote3', 'Tuote2', 'Tuote1', 'Tuote4','Tuote1','Tuote2','Tuote3'],
           'userid': ['Maria', 'Maria', 'Maria', 'Maria', 'Timo', 'Timo', 'Timo', 'Timo', 'Arttu','Leo', 'Leo', 'Anna', 'Anna', 'Saara', 'Saara', 'Saara'],
           'rating': [1,1,5,5,5,5,2,2,5,5,1,2,4,1,1,5]}
# dictin kääntäminen dataframeksi
df = pd.DataFrame(ratings)
df


,itemid,userid,rating
0,Tuote1,Maria,1
1,Tuote2,Maria,1
2,Tuote3,Maria,5
3,Tuote4,Maria,5
4,Tuote1,Timo,5
5,Tuote2,Timo,5
6,Tuote3,Timo,2
7,Tuote4,Timo,2
8,Tuote1,Arttu,5
9,Tuote3,Leo,5


In [5]:
# x = df.groupby("itemid")[["rating"]].value_counts()
# x
x = df.groupby("itemid")["rating"].value_counts()
x


itemid  rating
Tuote1  1         2
        5         2
        2         1
Tuote2  1         3
        5         1
Tuote3  5         3
        2         1
Tuote4  2         1
        4         1
        5         1
Name: count, dtype: int64

In [6]:
x2 = df.groupby(["itemid","rating"]).value_counts()
x2

itemid  rating  userid
Tuote1  1       Maria     1
                Saara     1
        2       Anna      1
        5       Arttu     1
                Timo      1
Tuote2  1       Leo       1
                Maria     1
                Saara     1
        5       Timo      1
Tuote3  2       Timo      1
        5       Leo       1
                Maria     1
                Saara     1
Tuote4  2       Timo      1
        4       Anna      1
        5       Maria     1
Name: count, dtype: int64

In [7]:
x3 = df.groupby(["userid","itemid"]).value_counts()
x3

userid  itemid  rating
Anna    Tuote1  2         1
        Tuote4  4         1
Arttu   Tuote1  5         1
Leo     Tuote2  1         1
        Tuote3  5         1
Maria   Tuote1  1         1
        Tuote2  1         1
        Tuote3  5         1
        Tuote4  5         1
Saara   Tuote1  1         1
        Tuote2  1         1
        Tuote3  5         1
Timo    Tuote1  5         1
        Tuote2  5         1
        Tuote3  2         1
        Tuote4  2         1
Name: count, dtype: int64

In [8]:
# mean_rating = df.groupby("itemid")[["rating"]].mean()
# mean_rating.sort_values(by='rating', ascending=False)
mean_all = df["rating"].mean()
print(mean_all)
mean_rating = df.groupby("itemid")[["rating"]].mean()
mean_rating.sort_values(by='rating', ascending=False)



3.125


,rating
itemid,
Tuote3,4.250000
Tuote4,3.666667
Tuote1,2.800000
Tuote2,2.000000


## Mallinnus

Readerin rakentaminen

In [9]:
lukija = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[["userid", "itemid", "rating"]], lukija)
data

Trainssetin rakentaminen

In [10]:
# # Set all data as training set >> tämän voi käyttää, jos riittävästi dataa
# from surprise.model_selection import train_test_split
# trainset, testset = train_test_split(data, test_size=0.25)
# print(trainset)
# print(testset)

In [11]:
# Set all data as training set
trainset = data.build_full_trainset()
trainset

Algoritmin rakentaminen

In [12]:
# Build and train an algorithm.
# Tarjotaan trainset algoritmille

sim_options = {"user_based": True} # False compute similarities between items, True between customers

algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


Testaus

In [13]:
user_id = "Leo"
item_id = "Tuote3"

pred = algo.predict(user_id, item_id, r_ui=5, verbose=True) # r_ui = real ui value

user: Leo        item: Tuote3     r_ui = 5.00   est = 4.93   {'actual_k': 4, 'was_impossible': False}


In [14]:
user_id = "Timo"
item_id = "Tuote3"

pred = algo.predict(user_id, item_id, r_ui=2, verbose=True) # r_ui = real ui value

user: Timo       item: Tuote3     r_ui = 2.00   est = 2.53   {'actual_k': 4, 'was_impossible': False}


In [15]:
user_id = "Maria"
item_id = "Tuote1"

pred = algo.predict(user_id, item_id, r_ui=1, verbose=True) # r_ui = real ui value

user: Maria      item: Tuote1     r_ui = 1.00   est = 1.39   {'actual_k': 5, 'was_impossible': False}


In [16]:
testset = trainset.build_anti_testset()
preds = algo.test(testset)
# print(preds)

# helppolukuinen output
for i in preds:
    print (i)


user: Arttu      item: Tuote2     r_ui = 3.12   est = 4.58   {'actual_k': 3, 'was_impossible': False}
user: Arttu      item: Tuote3     r_ui = 3.12   est = 2.32   {'actual_k': 3, 'was_impossible': False}
user: Arttu      item: Tuote4     r_ui = 3.12   est = 2.32   {'actual_k': 3, 'was_impossible': False}
user: Leo        item: Tuote1     r_ui = 3.12   est = 1.14   {'actual_k': 3, 'was_impossible': False}
user: Leo        item: Tuote4     r_ui = 3.12   est = 4.79   {'actual_k': 2, 'was_impossible': False}
user: Anna       item: Tuote2     r_ui = 3.12   est = 1.47   {'actual_k': 3, 'was_impossible': False}
user: Anna       item: Tuote3     r_ui = 3.12   est = 4.65   {'actual_k': 3, 'was_impossible': False}
user: Saara      item: Tuote4     r_ui = 3.12   est = 4.55   {'actual_k': 3, 'was_impossible': False}


In [18]:
pprint.pprint(preds)

[Prediction(uid='Arttu', iid='Tuote2', r_ui=3.125, est=4.578947368421052, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid='Arttu', iid='Tuote3', r_ui=3.125, est=2.31578947368421, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid='Arttu', iid='Tuote4', r_ui=3.125, est=2.3248730964467, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid='Leo', iid='Tuote1', r_ui=3.125, est=1.1428571428571428, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid='Leo', iid='Tuote4', r_ui=3.125, est=4.793103448275862, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid='Anna', iid='Tuote2', r_ui=3.125, est=1.4705882352941175, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid='Anna', iid='Tuote3', r_ui=3.125, est=4.647058823529412, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid='Saara', iid='Tuote4', r_ui=3.125, est=4.55072463768116, details={'actual_k': 3, 'was_impossible': False})]


Allaolevan käyttötarkoitus jää vielä epäselväksi...

In [ ]:
# This block copied from Surprise documentation at
# http://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(preds, n=10)#

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])



Arttu ['Tuote2', 'Tuote4', 'Tuote3']
Leo ['Tuote4', 'Tuote1']
Anna ['Tuote3', 'Tuote2']
Saara ['Tuote4']
Arttu 

('Tuote2', 4.578947368421052)
('Tuote4', 2.3248730964467)
('Tuote3', 2.31578947368421)
Leo 

('Tuote4', 4.793103448275862)
('Tuote1', 1.1428571428571428)
Anna 

('Tuote3', 4.647058823529412)
('Tuote2', 1.4705882352941175)
Saara 

('Tuote4', 4.55072463768116)


In [19]:
cross_validate(algo, data, measures=["MAE", "RMSE"], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE, RMSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MAE (testset)     1.1066  1.6121  1.9444  1.5544  0.3445  
RMSE (testset)    1.2415  2.0540  2.3366  1.8774  0.4642  
Fit time          0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    


{'test_mae': array([1.10658307, 1.61212121, 1.94444444]),
 'test_rmse': array([1.2415267 , 2.05400011, 2.33663787]),
 'fit_time': (0.0, 0.0, 0.0),
 'test_time': (0.0, 0.0, 0.0)}